# FCND Estimation Writeup

This writeup includes statements and supporting figures to explain how each rubric item was addressed, and specifically where in the code each step was handled.

## Implement Estimator

### Criterion 1: standard deviation

This criterion is to determine the standard deviation of the measurement noise of both GPS $X$ data and Accelerometer $X$ data. The method used for determining the standard deviation is based on Python `numpy.std` function as below.

```python
import numpy as np
gps_x = np.loadtxt('./config/log/Graph1.txt',delimiter=',',dtype='Float64',skiprows=1)[:,1]
acc_x = np.loadtxt('./config/log/Graph2.txt',delimiter=',',dtype='Float64',skiprows=1)[:,1]

gps_x_std = np.std(gps_x)
print(f'GPS X Std: {gps_x_std}')
acc_x_std = np.std(acc_x)
print(f'Accelerometer X Std: {acc_x_std}')
```

The results are:

```
GPS X Std: 0.743839343462462
Accelerometer X Std: 0.48770309814045426
```

Specifically, the calculated standard deviation should correctly capture ~$68\%$ of the sensor measurements. The output information is:

```
(../config/06_SensorNoise.txt)
PASS: ABS(Quad.GPS.X-Quad.Pos.X) was less than MeasuredStdDev_GPSPosXY for 69% of the time
PASS: ABS(Quad.IMU.AX-0.000000) was less than MeasuredStdDev_AccelXY for 67% of the time
```

<img src="fig6.png">

### Criterion 2: Implement a better rate gyro attitude integration scheme 

The purpose of this section is to implement a better rate gyro attitude integration scheme in the `UpdateFromIMU()` function. That means the improved integration scheme should result in an attitude estimator of < 0.1 rad for each of the Euler angles for a duration of at least 3 seconds during the simulation. The integration scheme could use at least two methods to improve performance over the current simple integration scheme. One is to use [quaternions(sec. 7.1.2)](https://www.overleaf.com/read/vymfngphcccj), another is a nonlinear transformation from gyro to the derivatives of Euler angles. The rotation matrix is:

$$rot=\begin{pmatrix}
1 & \sin\phi \tan\theta & \cos\phi \tan\theta \\
0 &  \cos\phi & -\sin\phi \\
0 &\sin\phi \sec\theta &  \cos\phi \sec\theta
\end{pmatrix}$$

Then, the derivatives of Euler angles are

$$rot\times gyro $$

The portion of `UpdateFromIMU()` function is:

```c
////////////////////////////// BEGIN STUDENT CODE ///////////////////////////
  // SMALL ANGLE GYRO INTEGRATION:
  // (replace the code below)
  // make sure you comment it out when you add your own code -- otherwise e.g. you might integrate yaw twice

    /*
  float predictedPitch = pitchEst + dtIMU * gyro.y;
  float predictedRoll = rollEst + dtIMU * gyro.x;
  ekfState(6) = ekfState(6) + dtIMU * gyro.z;	// yaw

  // normalize yaw to -pi .. pi
  if (ekfState(6) > F_PI) ekfState(6) -= 2.f*F_PI;
  if (ekfState(6) < -F_PI) ekfState(6) += 2.f*F_PI;
     */

    float phi = rollEst;
    float theta = pitchEst;
    
    Mat3x3F rot = Mat3x3F::Zeros();
    float sec=1/cos(theta);
    rot(0,0) = 1;
    rot(0,1) = sin(phi) * tan(theta);
    rot(0,2) = cos(phi) * tan(theta);
    rot(1,1) = cos(phi);
    rot(1,2) = -sin(phi);
    rot(2,1) = sin(phi) *sec;
    rot(2,2) = cos(phi) *sec;
    
    V3F Euler_angle_dot = rot * gyro;
    
    float predictedRoll = rollEst + dtIMU * Euler_angle_dot.x;
    float predictedPitch = pitchEst + dtIMU * Euler_angle_dot.y;
    ekfState(6) = ekfState(6) + dtIMU * Euler_angle_dot.z;
    
    // normalize yaw to -pi .. pi
    if (ekfState(6) > F_PI) ekfState(6) -= 2.f*F_PI;
    if (ekfState(6) < -F_PI) ekfState(6) += 2.f*F_PI;
/////////////////////////////// END STUDENT CODE ////////////////////////////

```

Then, the output is:

```
(../config/07_AttitudeEstimation.txt)
PASS: ABS(Quad.Est.E.MaxEuler) was less than 0.100000 for at least 3.000000 seconds
``` 

<img src="fig7.png">

### Criterion 3: Implement all of the elements of the prediction step for the estimator.

The prediction step includes the state update element (`PredictState()` function), a calculation of the Rgb prime matrix, and an update of the state covariance. The acceleration can be accounted for as a command in the calculation of gPrime. The covariance update should follow the classic EKF update equation. The following program is a portion of the function `PredictState()`, which is based on some basic physics:

```c
////////////////////////////// BEGIN STUDENT CODE ///////////////////////////
    // This task only use some basic physics, the dynamical model of the drone,
    // to predict the states of the vehicle.    
    predictedState(0) = curState(0) + dt * curState(3);
    predictedState(1) = curState(1) + dt * curState(4);
    predictedState(2) = curState(2) + dt * curState(5);
    
    V3F acc_w = attitude.Rotate_BtoI(accel); // a 3D vector
    
    predictedState(3) = curState(3) + dt * acc_w.x;
    predictedState(4) = curState(4) + dt * acc_w.y;
    predictedState(5) = curState(5) + dt * acc_w.z - dt * CONST_GRAVITY;
/////////////////////////////// END STUDENT CODE ////////////////////////////
```

After the `PredictState()` is finished, we can see the estimator state track the actual state, with only reasonably slow drift, as shown in the following figure:

<img src="fig8.png">

Then, let's introduce a realistic IMU, one with noise. When we run the scenario 09_PredictionCov, we can see a small fleet of quadcopter all using our prediction code to integrate forward. There are two plots:

* The top graph shows 10 (prediction-only) position X estimates.
* The bottom graph shows 10 (prediction-only) velocity estimates.

In `QuadEstimatorEKF.cpp`, the partial derivative of the body-to-global rotation matrix in the function `GetRbgPrime()` was calculated:

```c
////////////////////////////// BEGIN STUDENT CODE ///////////////////////////
    float cosTheta = cos(pitch);
    float sinTheta = sin(pitch);
    
    float cosPhi = cos(roll);
    float sinPhi = sin(roll);
    
    float sinPsi = sin(yaw);
    float cosPsi = cos(yaw);
    
    RbgPrime(0,0) = - cosTheta * sinPsi;
    RbgPrime(0,1) = - sinPhi  * sinTheta * sinPsi - cosTheta * cosPsi;
    RbgPrime(0,2) = - cosPhi  * sinTheta * sinPsi + sinPhi   * cosPsi;
    
    RbgPrime(1,0) = cosTheta * cosPsi;
    RbgPrime(1,1) = sinPhi  * sinTheta * cosPsi - cosPhi * sinPsi;
    RbgPrime(1,2) = cosPhi  * sinTheta * cosPsi + sinPhi * sinPsi;

/////////////////////////////// END STUDENT CODE ////////////////////////////
```

Then, the rest of the prediction step (predict the state covariance forward) in `Predict()` was implemented also:

```c
////////////////////////////// BEGIN STUDENT CODE ///////////////////////////
    gPrime(0,3) = dt;
    gPrime(1,4) = dt;
    gPrime(2,5) = dt;
    
    gPrime(3, 6) = (RbgPrime(0) * accel).sum() * dt;
    gPrime(4, 6) = (RbgPrime(1) * accel).sum() * dt;
    gPrime(5, 6) = (RbgPrime(2) * accel).sum() * dt;
    
    ekfCov = gPrime * ekfCov * gPrime.transpose() + Q;
/////////////////////////////// END STUDENT CODE ////////////////////////////
```

The next step is to tune the `QPosXYStd` and the `QVelXYStd` process parameters in `QuadEstimatorEKF.txt`.

```
#QPosXYStd = .05
#QPosZStd = .05
QPosXYStd = .02
QPosZStd = .02

# QVelXYStd = .05
QVelXYStd = .2
```

The result is:

<img src="fig9.png">

### Criterion 4: Implement the magnetometer update.

Up until now, we've only used the accelerometer and gyro for our state estimation. In this step, we will be adding the information from the magnetometer to improve our filter's performance in estimating the vehicle's heading.

First, we tune the parameter `QYawStd` (QuadEstimatorEKF.txt) for the `QuadEstimatorEKF` so that it approximately captures the magnitude of the drift.

```
# QYawStd = .05
QYawStd = .1
```

Then, we implement magnetometer update in the function `UpdateFromMag()`. A resulting plot is shown as below:

<img src="fig10.png">

Our goal is to both have an estimated standard deviation that accurately captures the error and maintain an error of less than 0.1rad in heading for at least 10 seconds of the simulation. The result is:

```
(../config/10_MagUpdate.txt)
PASS: ABS(Quad.Est.E.Yaw) was less than 0.120000 for at least 10.000000 seconds
PASS: ABS(Quad.Est.E.Yaw-0.000000) was less than Quad.Est.S.Yaw for 69% of the time
```

### Criterion 5: Implement the GPS update.

1. Run scenario 11_GPSUpdate. At the moment this scenario is using both an ideal estimator and and ideal IMU. Even with these ideal elements, watch the position and velocity errors (bottom right). We can see they are drifting away, since GPS update is not yet implemented.

2. Let's change to using our estimator by setting `Quad.UseIdealEstimator` to 0 in `config/11_GPSUpdate.txt`.

```
# Quad.UseIdealEstimator = 1
Quad.UseIdealEstimator = 0
```

3. Now repeat with realistic IMU by commenting out these lines in config/11_GPSUpdate.txt:

```
# SimIMU.AccelStd = 0,0,0
# SimIMU.GyroStd = 0,0,0
```

4. Tune the process noise model in `QuadEstimatorEKF.txt` to try to approximately capture the error you see with the estimated uncertainty (standard deviation) of the filter.

5. Implement the EKF GPS Update in the function `UpdateFromGPS()`.

```c
// GPS UPDATE
  // Hints: 
  //  - The GPS measurement covariance is available in member variable R_GPS
  //  - this is a very simple update
  ////////////////////////////// BEGIN STUDENT CODE ///////////////////////////
    for (int i=0; i<6; i++){
        zFromX(i) = ekfState(i);
    }
    
    for ( int i = 0; i < 6; i++) {
        hPrime(i,i) = 1;
    }
/////////////////////////////// END STUDENT CODE ////////////////////////////
```

6. Now once again re-run the simulation. Our objective is to complete the entire simulation cycle with estimated position error of < 1m (we can see a green box over the bottom graph if we succeed). We may to try experimenting with the GPS update parameters to try and get better performance.

The output is:

```
(../config/11_GPSUpdate.txt)
PASS: ABS(Quad.Est.E.Pos) was less than 1.000000 for at least 20.000000 seconds
```

And, the resulting plot is:

<img src="fig11.png">

## Conclusions

For each step of the project, the final estimator can be able to successfully meet the performance criteria with the controller provided. The estimator's parameters could be properly adjusted to satisfy each of the performance criteria elements.

The controller developed in the previous project has been de-tuned to successfully meet the performance criteria of the final scenario (<1m error for entire box flight).